In [14]:
!pip install tensorflow
!pip install ucimlrepo

You should consider upgrading via the 'C:\Users\rodri\Documents\projects\iris-classification-ml\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [2]:
from ucimlrepo import fetch_ucirepo
from sklearn.model_selection import train_test_split
from google.cloud import storage
import pandas as pd
import numpy as np
import tensorflow as tf

In [6]:
PROJECT = "iris-classification-404414"
BUCKET = "iris-classification-bucket"
REGION = "southamerica-east1"
OUTDIR = f"gs://{BUCKET}/iris/data"

In [7]:
%env PROJECT=$PROJECT
%env BUCKET=$BUCKET
%env REGION=$REGION
%env OUTDIR=$OUTDIR
%env TFVERSION=2.14

env: PROJECT=iris-classification-404414
env: BUCKET=iris-classification-bucket
env: REGION=southamerica-east1
env: OUTDIR=gs://iris-classification-bucket/iris/data
env: TFVERSION=2.14


In [13]:
!gcloud config set project $PROJECT


To update your Application Default Credentials quota project, use the `gcloud auth application-default set-quota-project` command.
Updated property [core/project].


In [3]:
# fetch dataset
iris = fetch_ucirepo(id=53)

# data (as pandas dataframes)
X = iris.data.features
y = iris.data.targets
_, y_train = np.unique(y, return_inverse=True)
y_train = tf.keras.utils.to_categorical(y_train, num_classes=3)
result = pd.concat([X, y], axis=1)

train, eval = train_test_split(result, test_size=0.2)
result.head()

,sepal length,sepal width,petal length,petal width,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [5]:
result.columns

Index(['sepal length', 'sepal width', 'petal length', 'petal width', 'class'], dtype='object')

In [ ]:
client = storage.Client()
bucket = client.get_bucket(BUCKET)

In [26]:
bucket.blob('input/train.csv').upload_from_string(train.to_csv(index=False), 'text/csv')
bucket.blob('input/eval.csv').upload_from_string(eval.to_csv(index=False), 'text/csv')

In [ ]:
!python ./iris/setup.py sdist --formats=gztar